In [2]:
%config IPCompleter.greedy=True

In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import ipywidgets
from IPython.display import display
from time import sleep

In [7]:
#filename = "solidYellowLeft.mp4"
filename = "solidWhiteRight.mp4"
#filename = "challenge.mp4"

#open the video source
cap = cv2.VideoCapture("test_videos/" + filename)

#get video parameters
fps = cap.get(cv2.CAP_PROP_FPS)
imshape = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))

#set up video output
out = cv2.VideoWriter("test_videos_output/" + filename[:-4] + "_out.avi", cv2.VideoWriter_fourcc('H','2','6','4'), fps, (imshape[1], imshape[0]))

#initialize image widget
image_widget = ipywidgets.Image(format='jpeg')
image_widget.width = 500
display(image_widget)

#main video processing loop
while cap.isOpened():
    
    #read the next frame
    ret, image = cap.read()
    
    #break the loop on EOF
    if ret == 0:
        break
    
    #create the grayscale version of the frame
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    #blur the grayscale image
    kernel_size = 5
    blur_gray = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
    
    #Canny edge detection
    low_thr = 50
    high_thr = 150
    edges = cv2.Canny(blur_gray, low_thr, high_thr)
    
     
    #plt.imshow(edges, cmap='gray')
    mask = np.zeros_like(edges)
    ignore_mask_color = 255
    
    #define the mask shape
    #imshape = edges.shape
    maskBottomLeft = (0,imshape[0])
    maskTopLeft = (int(imshape[1]/2-imshape[1]/30),int(imshape[0]/2+imshape[0]/10))
    maskTopRight = (int(imshape[1]/2+imshape[1]/30),int(imshape[0]/2+imshape[0]/10))
    maskBottomRight = (imshape[1],imshape[0])
    vertices = np.array([[maskBottomLeft, maskTopLeft, maskTopRight, maskBottomRight]], dtype=np.int32)
    
    #create the mask
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #apply the mask to detected edges
    masked_edges = cv2.bitwise_and(edges, mask)

    #Hough transform
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15 #35     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 25 #minimum number of pixels making up a line
    max_line_gap = 20    # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)
    
    #iterating through detected lines
    leftSlope = []
    leftIntercept = []
    rightSlope = []
    rightIntercept = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            #watching out for division by zero
            if (x2-x1) != 0:
                #calculating line slope
                slope = (y2-y1)/(x2-x1)
                #process the left lane lines 
                if slope < -0.4:
                    slope, intercept = np.polyfit([x1, x2], [y1, y2], 1)
                    leftSlope.append(slope)
                    leftIntercept.append(intercept)
                #process the right lane lines
                if slope > 0.4:  
                    slope, intercept = np.polyfit([x1, x2], [y1, y2], 1)
                    rightSlope.append(slope)
                    rightIntercept.append(intercept)
            
    #define the left lane and right lane functions if there were any lines detected
    if ((len(leftSlope) > 0) & (len(leftIntercept) > 0)):
        fxLeft = np.poly1d((np.mean(leftSlope), np.mean(leftIntercept)))
        #draw the lane lines
        cv2.line(line_image, (maskBottomLeft[0], int(fxLeft(maskBottomLeft[0]))), 
                 (maskTopLeft[0], int(fxLeft(maskTopLeft[0]))), (0,0,255), 10)
    if ((len(rightSlope) > 0) & (len(rightIntercept) >0)):
        fxRight = np.poly1d((np.mean(rightSlope), np.mean(rightIntercept)))
        #draw the lane lines
        cv2.line(line_image, (maskTopRight[0], int(fxRight(maskTopRight[0]))), 
                 (maskBottomRight[0], int(fxRight(maskBottomRight[0]))), (0,0,255), 10)
       
    #apply lines to the original image
    lines_img = cv2.addWeighted(image, 0.8, line_image, 1, 0)
    
    #update the image widget
    image_widget.value = cv2.imencode('.jpg', lines_img)[1].tostring()
    #write output frame
    out.write(lines_img)

#release input and output
cap.release()
out.release()

Image(value=b'', format='jpeg', width='500')